# Quantum Annealing Method for Dynamic Virtual Machine and Task Allocation in Cloud Infrastructures from Sustainability Perspective

   #### Valter Uotila, Jiaheng Lu
    Unified Database Management Systems research group
    Department of Computer Science
    University of Helsinki
    Finland

The initial version of this work won the Most Creative Open Hackathon Experiment on Braket in Amazon Braket Challenge and Third place in IBM Qiskit Challenge at the QHack quantum computing hackathon organized by Xanadu in the spring of 2022.

In [1]:
import dimod
from dimod.generators.constraints import combinations
from dwave.system import LeapHybridSampler
from hybrid.reference import KerberosSampler

import itertools
import math
import json

from flask import Flask
from flask import request

## Useful functions

In [2]:
def construct_variables(datacenter_state, previous_state = []):
    variables = set()
    for task in datacenter_state["cloudlets"]:
        for vm in datacenter_state["vms"]:
            if previous_state != []:
                for var in previous_state:
                    if int(task) == int(var[0]) and int(vm) == int(var[1]):
                        for pm in datacenter_state["hosts"]:
                            variables.add((int(task), int(vm), int(pm)))
            else:
                for pm in datacenter_state["hosts"]:
                    variables.add((int(task), int(vm), int(pm)))
    return list(variables)

In [3]:
def group_by(variables, index):
    groups = {}
    for var in variables:
        if var[index] in groups:
            groups[var[index]].append(var)
        else:
            groups[var[index]] = [var]
    return groups

In [4]:
def construct_previous_state(datacenter_state, first_run):
    vm_pm_state = []
    state = []
    if not first_run:
        for vm in datacenter_state["vms"]:
            vm_pm_state.append((vm, datacenter_state["vms"][vm]["hostid"]))
        for cloudlet in datacenter_state["cloudlets"]:
            cloudlet_id = datacenter_state["cloudlets"][cloudlet]["cloudletId"]
            cloudlet_vm = datacenter_state["cloudlets"][cloudlet]["vmId"]
            for vm_pm in vm_pm_state:
                if int(vm_pm[0]) == int(cloudlet_vm):
                    state.append((cloudlet_id, vm_pm[0], vm_pm[1]))
    return state

In [5]:
def print_solution(sample):
    for varname, value in sample.items():
        if value == 1:
            print(varname, value)

In [6]:
def construct_migration_map(solution):
    migration_map = {}
    for varname, value in solution.items():
        if value == 1:
            migration_map[str(varname[1])] = str(varname[2]) 
    return migration_map

In [7]:
def construct_task_allocation_map(solution):
    task_allocation_map = {}
    for varname, value in solution.items():
        if value == 1:
            task_allocation_map[str(varname[0])] = str(varname[1]) 
    return task_allocation_map

## Metrics

These metrics can be modified based on the cloud infrastructure.

In [8]:
def price(t, v, datacenter_state):
    return 0

In [9]:
def get_task_mips(task_id, datacenter_state):
    return datacenter_state["cloudlets"][str(task_id)]["totalLength"]

def get_vm_mips(vm_id, datacenter_state):
    return datacenter_state["vms"][str(vm_id)]["mips"]

def get_pm_mips(pm_id, datacenter_state):
    return datacenter_state["hosts"][str(pm_id)]["mips"]

def get_pm_power(pm_id, datacenter_state):
    return datacenter_state["hosts"][str(pm_id)]["power"]

def get_vm_bw(vm_id, datacenter_state):
    return datacenter_state["vms"][str(vm_id)]["bw"]

def get_pm_bw(pm_id, datacenter_state):
    return datacenter_state["hosts"][str(pm_id)]["availableBw"]

def get_sustainability_scaling_value(pm_id):
    return pm_id**2

def get_sustainability_offset(pm_id):
    return pm_id**3

In [10]:
def sustainability(t, v, m, datacenter_state):
    # (VM mips / PM MIPS)*max power for PM*sustainability scaling value for power production + carbon footprint offset
    power =  get_pm_power(m, datacenter_state)
    v_mips = get_vm_mips(v, datacenter_state)
    m_mips = get_pm_mips(m, datacenter_state)
    
    scaling_value = get_sustainability_scaling_value(m)
    offset = get_sustainability_offset(m)
    
    carbon_footprint = (v_mips / m_mips)*power*scaling_value + offset
    
    return carbon_footprint

In [11]:
def migration_cost(to_var, from_var, datacenter_state):
    # Determining migration cost is a bit open question in the implementation
    # Energy and carbon footprint
    # Time and resources
    # VM size / available bandwidth
    
    if to_var == from_var:
        return 0
    
    cost = 0
    vm_bw = get_vm_bw(from_var[1], datacenter_state)
    pm_bw = get_pm_bw(to_var[2], datacenter_state)
    
    if vm_bw > pm_bw:
        cost = 1000000
    else:
        cost = pm_bw - vm_bw
    
    return cost

## Constraints

### Hard constraints

Violation of these constraints is not technically possible and these constraints should have high priority in order that the solution is even valid. These constraints are forced to be true by adjust the `strength` constant.

#### Constraint: every task executed on a single virtual machine

In [12]:
def every_task_executed_on_single_vm(variables, bqm):
    strength = 10000000
    groups = group_by(variables, 0)
    for group in groups:
        combs = combinations(groups[group], 1, strength)
        bqm.update(combs)
    return bqm

#### Constraint: every virtual machine runs on single physical machine

In [13]:
def every_vm_runs_on_single_pm(variables, bqm):
    strength = 10000000
    groups = group_by(variables, 1)
    for group in groups:
        combs = combinations(groups[group], 1, strength)
        bqm.update(combs)
    return bqm

### Soft constraints

Violation of these constraints is possible but not desired. The mimization of these constraints makes the solution more optimal.

#### Constraint: The available virtual machines should be utilized efficiently 

Since cloudlet MIPS > VM MIPS, we aim to position cloudlets to virtual machines so that cloudlets' total running time is minimized.

In [14]:
def maximize_vm_utilization(variables, bqm, datacenter_state):
    vms = group_by(variables, 1)
    linear = dict()
    quadratic = dict()
    offset = 0.0
    A = 2
    
    for vm in vms:
        quadratic_terms = itertools.combinations(vms[vm], 2)
        total_for_this_vm = get_vm_mips(vm, datacenter_state)
        offset += total_for_this_vm**2
        
        # Quadratic terms
        for t in quadratic_terms:
            task1, task2 = t[0][0], t[1][0]
            quadratic[t] = 2*get_task_mips(task1, datacenter_state)*get_task_mips(task2, datacenter_state)
            
        # Linear terms
        for var in vms[vm]:
            task = var[0]
            linear[var] = get_task_mips(task, datacenter_state)**2 - 2*total_for_this_vm
            
    bqm_migration = dimod.BinaryQuadraticModel(linear, quadratic, offset, dimod.BINARY)
    bqm_migration.scale(A)
    bqm.update(bqm_migration)
    return bqm

#### Constraint: The available physical machines should be utilized efficiently

This step aims to optimize the virtual machine allocation by allocating virtual machines to physical machines so that the execution time is minimized. The execution time is estimated by considering task sizes on virtual machines in MIPS and considering virtual machine sizes on physical machines in MIPS. When the physical resources are utilized efficiently, the running time is minimized.

It is important that VM MIPS < PM MIPS. If the condition is not satisfied, no VM is allocated to a PM.

In [15]:
def maximize_pm_utilization(variables, bqm, datacenter_state):
    pms = group_by(variables, 2)
    linear = dict()
    quadratic = dict()
    offset = 0.0
    A = 2
    
    for p in pms:
        quadratic_terms = itertools.combinations(pms[p], 2)
        total_for_this_pm = get_pm_mips(p, datacenter_state)
        offset += total_for_this_pm**2

        # Quadratic terms
        for t in quadratic_terms:
            vm1, vm2 = t[0][1], t[1][1]
            quadratic[t] = 2*get_vm_mips(vm1, datacenter_state)*get_vm_mips(vm2, datacenter_state)

        # Linear terms
        for var in pms[p]:
            vm = var[1]
            linear[var] = get_vm_mips(vm, datacenter_state)**2 - 2*total_for_this_pm
    
    bqm_migration = dimod.BinaryQuadraticModel(linear, quadratic, offset, dimod.BINARY)
    bqm_migration.scale(A)
    bqm.update(bqm_migration)
    return bqm

#### Constraint: migration cost between machines should be minimized

Virtual machine allocations after the initial step can be migrated between the physical machines in the cloud infrastructure.

In [16]:
def minimize_migration_cost(variables, previous_state, bqm, datacenter_state):
    A = 20
    linear = dict()
    quadratic = dict()
    to_vms = group_by(variables, 1)
    from_vms = previous_state

    for from_vm in from_vms:
        for to_vm in to_vms:
            for to_vm_var in to_vms[to_vm]:
                linear[to_vm_var] = migration_cost(to_vm_var, from_vm, datacenter_state)
    
    bqm_migration = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, dimod.BINARY)
    bqm_migration.scale(A)
    bqm.update(bqm_migration)
    return bqm

#### Constraint: power consumption should be minimized

Since previous research has especially focused on power consumption (instead of sustainability), we implement also a constraint to minimize it. This makes the implementation comparable to the power-aware VM allocation algorithms implemented in CloudSim. 

In [17]:
def minimize_power_consumption(variables, bqm, datacenter_state):
    A = 1
    linear = dict()
    for var in variables:
        total_power = get_pm_power(var[2], datacenter_state)
        vm_mips = get_vm_mips(var[1], datacenter_state)
        pm_mips = get_pm_mips(var[2], datacenter_state)
        linear[var] = total_power*(vm_mips/pm_mips)
        
    bqm_migration = dimod.BinaryQuadraticModel(linear, dict(), 0.0, dimod.BINARY)
    bqm.scale(A)
    bqm.update(bqm_migration)
    return bqm

#### Constraint: sustainability should be maximized

In [18]:
def maximize_sustainability(variables, bqm, datacenter_state):
    A = 1
    linear = dict()
    for var in variables:
        linear[var] = sustainability(var[0], var[1], var[2], datacenter_state)
        
    bqm_migration = dimod.BinaryQuadraticModel(linear, dict(), 0.0, dimod.BINARY)
    bqm.scale(A)
    bqm.update(bqm_migration)
    return bqm

## Optimization on quantum annealer

In [19]:
def initialize():
    vartype = dimod.BINARY
    linear = dict()
    quadratic = dict()
    offset = 0.0
    bqm = dimod.BinaryQuadraticModel(linear, quadratic, offset, vartype)
    return bqm

In [20]:
def solve(bqm, device = "Simulated"):
    sample = None
    bqm.normalize()
    if device == "Simulated":
        sampler = dimod.SimulatedAnnealingSampler()
        sampleset = sampler.sample(bqm, num_reads=100)
        sample = sampleset.first.sample
    elif device == "Kerberos":
        kerberos_sampler = KerberosSampler().sample(bqm, max_iter=10, convergence=3, qpu_params={'label': 'Task and VM allocation'})
        sample = kerberos_sampler.first.sample
    elif device == "LeapHybrid":
        sampler = LeapHybridSampler()
        sampleset = sampler.sample(bqm)
        sample = sampleset.first.sample
    return sample

In [21]:
app = Flask(__name__)

@app.route("/", methods=['POST'])
def optimize_allocation():
    datacenter_state = request.get_json()
    time = datacenter_state["time"]
    interval = datacenter_state["interval"]
    first_run = time == 0.0
    
    previous_state = construct_previous_state(datacenter_state, first_run)
    
    variables = construct_variables(datacenter_state, previous_state)
    
    bqm = initialize()
    
    print("The problem contains", len(variables), "variables.")
    
    print("Constructing qubo...")
    bqm = every_task_executed_on_single_vm(variables, bqm)
    bqm = every_vm_runs_on_single_pm(variables, bqm)
    bqm = maximize_pm_utilization(variables, bqm, datacenter_state)
    
    bqm = minimize_power_consumption(variables, bqm, datacenter_state)
    #bqm = maximize_sustainability(variables, bqm, datacenter_state)
    
    # We optimize the task allocation on virtual machines on the first optimization round
    # After that we minimize migration cost of virtual machines
    if first_run:
        bqm = maximize_vm_utilization(variables, bqm, datacenter_state)
    else:
        bqm = minimize_migration_cost(variables, previous_state, bqm, datacenter_state)
    
    print("Number of quadratic terms", len(bqm.quadratic))
    #for q in list(bqm.quadratic)[:20]:
    #    print(bqm.quadratic[q])
    
    device = "LeapHybrid"
    if len(variables) < 100:
        device = "Simulated"
    print("Solving QUBO on", device)
    solution = solve(bqm, device)
    print_solution(solution)
    
    response = construct_migration_map(solution)
    
    if first_run:
        response = construct_task_allocation_map(solution)
        
    #response = {}
    #for v in datacenter_state["vms"]:
    #    response[v] = v
    
    return response

In [ ]:
app.run(port=1234)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1234/ (Press CTRL+C to quit)


The problem contains 8000 variables.
Constructing qubo...
Number of quadratic terms 4560000
Solving QUBO on LeapHybrid


[2023-01-12 13:05:54,023] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\valte\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\valte\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\valte\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\valte\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\valte\AppData\Local\Temp\ipykernel_26752\1923799858.py", line 41, in optimize_allocation
    solution = solve(bqm, device)
  File "C:\Users\valte\AppData\Local\Temp\ipykernel_2675

The solution to the optimization problem is a new allocation of tasks into virtual machines and virtual machines to physical machines. This allocation is represented as a migration map. Unfortunately CloudSim does not implement reallocation of tasks during running simulation so task-virtual machine allocation stays the same after the initial allocation. Thus the migration map describes just virtual machine to physical machine allocation.